# jovian-ml-project-rough

Use the "Run" button to execute the code.

In [84]:
#pip install jovian --upgrade --quiet

In [85]:
#import jovian

In [86]:
# Execute this to save new versions of the notebook
#jovian.commit(project="jovian-ml-project",filename="jovian-ml-project")

In [183]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import xml.etree.ElementTree as ET
import time
import re
from tqdm import tqdm
%matplotlib inline

In [88]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.simplefilter(action='ignore')

In [184]:
tqdm.pandas(desc="my bar!")

<a id='top'>**Beginning of Notebook**</a>

[model train text data](#train_data)

#### train data

In [89]:
train_text_df = pd.read_csv('training_text.zip', sep="\|\|", encoding="utf-8",engine="python", skiprows=1, names=["ID", "Text"],compression='zip')

In [90]:
train_vari_df = pd.read_csv('training_variants.zip',compression='zip')

In [91]:
train_vari_df['Class'].value_counts()

7    953
4    686
1    568
2    452
6    275
5    242
3     89
9     37
8     19
Name: Class, dtype: int64

In [92]:
train_text_df.shape, train_vari_df.shape

((3321, 2), (3321, 4))

In [93]:
train_text_df.columns, train_vari_df.columns

(Index(['ID', 'Text'], dtype='object'),
 Index(['ID', 'Gene', 'Variation', 'Class'], dtype='object'))

In [94]:
# merging the text and variation data
train_df = pd.merge(train_vari_df,train_text_df,on='ID')

In [95]:
# taking a sample of the dataset
train_df = train_df.sample(frac=0.2)

In [96]:
train_df.head()

,ID,Gene,Variation,Class,Text
2631,2631,BRCA1,L1780P,4,Abstract The BRCA1 gene from individuals at ...
2261,2261,PTEN,F154L,4,ABSTRACT To look for a direct role of ultravi...
2954,2954,GNAS,Q227L,2,A subset of growth hormone-secreting human pit...
435,435,TP53,R249S,1,Many p53 missense mutations possess dominant-n...
1101,1101,BAP1,C91S,4,The deubiquitinating enzyme BRCA1-associated p...


In [97]:
train_df.shape

(664, 5)

In [98]:
train_df['Class'].value_counts()

7    182
4    144
1    115
2     89
6     63
5     40
3     21
9      7
8      3
Name: Class, dtype: int64

In [99]:
test_text_df = pd.read_csv('test_text.zip', sep="\|\|", encoding="utf-8",engine="python", skiprows=1, names=["ID", "Text"],compression='zip')


In [100]:
test_vari_df  = pd.read_csv('test_variants.zip',compression='zip')

In [101]:
test_vari_df.head()

,ID,Gene,Variation
0,0,ACSL4,R570S
1,1,NAGLU,P521L
2,2,PAH,L333F
3,3,ING1,A148D
4,4,TMEM216,G77A


In [102]:
test_vari_df.shape

(5668, 3)

In [103]:
# test_vari_df sample
test_vari_df=test_vari_df.sample(frac=0.1)

In [104]:
# test_text_df sample
test_text_df = test_text_df.sample(frac=0.1)

In [105]:
test_text_df.shape

(567, 2)

In [106]:
test_text_df.head()

,ID,Text
3767,3767,It the Fl C329Y rate for Deletions cells and a...
3185,3185,The mitotic kinase Aurora A (Aur-A) is require...
1291,1291,Aggressive systemic mastocytosis (ASM) is a ve...
2155,2155,Mutations of the sequence-specific master regu...
4114,4114,Introduction Somatic mutations in human cytoso...


In [107]:
train_vari_df.groupby('Gene').sum().head()

,ID,Class
Gene,,
ABL1,22087,62
ACVR1,3207,16
AGO2,10435,6
AKT1,37898,165
AKT2,15103,76


In [108]:
train_vari_df.groupby('Gene').count().head()

,ID,Variation,Class
Gene,,,
ABL1,26,26,26
ACVR1,3,3,3
AGO2,5,5,5
AKT1,28,28,28
AKT2,11,11,11


In [109]:
train_vari_df[train_vari_df['Gene'] == 'AGO2']

,ID,Gene,Variation,Class
2085,2085,AGO2,R812A,1
2086,2086,AGO2,R647A,1
2087,2087,AGO2,R583A,1
2088,2088,AGO2,Amplification,2
2089,2089,AGO2,Q545A,1


In [110]:
merged_df=train_vari_df.merge(train_text_df,on= 'ID')

[link to destination](#destination)

In [111]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [112]:
revised_punctuation='!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'

In [113]:
revised_punctuation

'!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~'

In [114]:
#!pip install nltk

In [115]:
import nltk
nltk.download('words')
from nltk.corpus import words
words=nltk.corpus.words.words()

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\anubr\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [116]:
len(set(words))

235892

In [117]:
nltk_words=set(words)

In [118]:
# set did not help as it reduces the number to 
# def reduce_words(row,index): 
#     text=row[1]
#     if isinstance(text,str):
#         new_text_words = set(text.split())
        
#         important_words = new_text_words.difference(words)
#         edited_words = " ".join(important_words)
        
#         intersected_words = new_text_words.intersection(words)
#         edit_intersect_words = " ".join(intersected_words)
#         train_text_df['new_text'][index] = edited_words
#         train_text_df['intsct_text'][index] = edit_intersect_words 

In [119]:
#from collections import Counter

In [120]:
def reduce_words_df(row,index): 
    text=row[1]
    if isinstance(text,str):
        new_text_words = text.split()
        counted_words = Counter(new_text_words) 
        counted_words=sorted(counted_words.items(),key=lambda x:x[1], reverse=True)
        test['new_text'][index] = counted_words        

In [121]:
# for index,row in test.iterrows():
#     reduce_words(row,index)

In [122]:
def cleanse_data(data):        
    if isinstance(data,str):
        no_punctuation = [char for char in data.replace('\n',' ') if char not in revised_punctuation]
        no_punctuation_txt = ''.join(no_punctuation)                   
        return no_punctuation_txt
    else:
        return "NA"

In [123]:
def reduce_words(data):
    data = data.split()
    words = [word for word in data if word not in nltk_words]
    text = " ".join(words)
    return text    

In [124]:
def word_count(text):
    text_lst=text.split()
    vector = [text_lst.count(str(word)) for word in text_lst]
    return vector[:300]

In [125]:
from nltk.probability import FreqDist

we have obtained the frequency distribution of words using freqdist

In [126]:
def freq_dist(data):
    data=data.split(" ")
    freq=FreqDist(data) # gets word frequency
    freq=sorted(freq.items(),key=lambda x:x[1], reverse=True) # sorts words from highest to lowest frequency
    return [item[1] for item in freq][:300] # returns a fixed length of 300 words

In [127]:
# implementing bag of words

In [128]:
from sklearn.feature_extraction.text import CountVectorizer

In [129]:
bow_vectorizer = CountVectorizer(stop_words='english',max_features=1600)

In [130]:
# implementing tf-idf

In [131]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [132]:
tfidf_vectorizer = TfidfVectorizer(
    min_df=1, max_features=1600, strip_accents='unicode',lowercase =True,
    analyzer='word', token_pattern=r'\w+', ngram_range=(1, 3), use_idf=True, 
    smooth_idf=True, sublinear_tf=True, stop_words = 'english')

In [133]:
# https://melaniewalsh.github.io/Intro-Cultural-Analytics/05-Text-Analysis/03-TF-IDF-Scikit-Learn.html
# https://www.datacamp.com/community/tutorials/understanding-logistic-regression-python
# https://machinelearningmastery.com/multinomial-logistic-regression-with-python/
# https://www.kaggle.com/elemento/personalizedmedicine-rct

In [134]:
from sklearn.linear_model import LogisticRegression

In [135]:
from sklearn.metrics import log_loss

In [136]:
# define the multinomial logistic regression model
bow_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
tfidf_model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

In [137]:
#sorted(bow_model.coef_.tolist()[0])[:5]

In [138]:
#sorted(tfidf_model.coef_.tolist()[0])[:5]

<a id="train_data" style="font-size:22px">model train text sample data</a>

[Beginning of Notebook](#top)

### handling the train text sample data                  

#### text preprocessing

In [139]:
train_df.head()

,ID,Gene,Variation,Class,Text
2631,2631,BRCA1,L1780P,4,Abstract The BRCA1 gene from individuals at ...
2261,2261,PTEN,F154L,4,ABSTRACT To look for a direct role of ultravi...
2954,2954,GNAS,Q227L,2,A subset of growth hormone-secreting human pit...
435,435,TP53,R249S,1,Many p53 missense mutations possess dominant-n...
1101,1101,BAP1,C91S,4,The deubiquitinating enzyme BRCA1-associated p...


In [140]:
# remove punctuation
train_df['cleaned_text'] = train_df['Text'].map(cleanse_data,na_action='ignore')

In [141]:
# filtering all common english words
train_df['cleaned_text'] = train_df['cleaned_text'].map(reduce_words,na_action='ignore')

In [142]:
train_df['Text_len'] = train_df['Text'].map(lambda x:len(str(x)))
train_df['cleaned_text_len'] = train_df['cleaned_text'].map(lambda x:len(str(x)))

In [143]:
train_df.head()

,ID,Gene,Variation,Class,Text,cleaned_text,Text_len,cleaned_text_len
2631,2631,BRCA1,L1780P,4,Abstract The BRCA1 gene from individuals at ...,Abstract BRCA1 individuals cancers mutations H...,96571,36917
2261,2261,PTEN,F154L,4,ABSTRACT To look for a direct role of ultravi...,ABSTRACT To UV pigmentosum XP patients DNA 100...,20640,8748
2954,2954,GNAS,Q227L,2,A subset of growth hormone-secreting human pit...,hormone-secreting tumours carries mutations GT...,30426,12801
435,435,TP53,R249S,1,Many p53 missense mutations possess dominant-n...,Many p53 missense mutations dominant-negative ...,135119,55749
1101,1101,BAP1,C91S,4,The deubiquitinating enzyme BRCA1-associated p...,deubiquitinating BRCA1-associated 1 BAP1 posse...,87723,38661


In [147]:
train_df[train_df['cleaned_text'].map(lambda x:not isinstance(x,str))]

,ID,Gene,Variation,Class,Text,cleaned_text,Text_len,cleaned_text_len


In [146]:
train_df.drop([1109],axis=0,inplace=True)

#### converting text to numbers

In [148]:
train_df_corpus = train_df['cleaned_text'].tolist()

In [149]:
bow_vectorizer.fit(train_df_corpus)

CountVectorizer(max_features=1600, stop_words='english')

In [150]:
train_df_counts= bow_vectorizer.transform(train_df_corpus)

In [152]:
print("Vocabulary: ", bow_vectorizer.vocabulary_)

Vocabulary:  {'brca1': 329, 'individuals': 812, 'cancers': 348, 'mutations': 1011, 'alleles': 233, 'missense': 976, 'counseling': 450, 'cancer': 346, 'associated': 279, 'dimensional': 517, '37': 111, 'terms': 1467, 'loss': 906, 'function': 671, 'changes': 375, 'terminus': 1466, 'brct': 332, 'localized': 898, 'forms': 663, 'structure': 1423, 'based': 297, 'predicting': 1176, 'consequences': 424, '57': 143, 'germ': 698, 'line': 892, 'variants': 1556, 'integrated': 832, 'introduction': 838, 'inherited': 818, 'allowed': 235, 'developing': 511, 'approaches': 264, 'identifying': 771, 'causing': 357, 'domains': 537, 'functions': 673, 'mutants': 1008, 'sequences': 1349, 'structures': 1424, 'studies': 1425, 'proteins': 1210, '10': 14, '11': 27, '12': 31, 'including': 803, '13': 37, '14': 41, 'activates': 210, 'dna': 533, '15': 43, '16': 45, 'cooh': 440, 'factors': 623, 'rna': 1305, 'polymerase': 1164, 'ii': 778, '17': 48, 'activating': 211, 'binding': 310, '18': 49, '19': 51, 'importantly': 798

In [153]:
X_train_bow = train_df_counts.toarray()

In [154]:
y_train = train_df['Class']

#### model fitting

In [155]:
train_df.shape

(664, 8)

In [156]:
from numpy import random

In [157]:
# create a dummy y value for getting the baseline accuracy
y_dummy = random.rand(664,9)

In [159]:
#y_dummy

In [160]:
# baseline loss metric value
log_loss(y_train,y_dummy)

2.429296232507611

#### Logistic Regression Model

In [161]:
bow_model.fit(X_train_bow,y_train)

LogisticRegression(multi_class='multinomial')

In [162]:
sorted(bow_model.coef_.tolist()[0])[:5]

[-0.1303172589801489,
 -0.1195262532408153,
 -0.10716254832255036,
 -0.10499755830069249,
 -0.10381004770354639]

In [163]:
y_pred_train_bow = bow_model.predict_proba(X_train_bow)

In [164]:
log_loss(y_train,y_pred_train_bow)

0.1392230048331143

In [165]:
tfidf_vectorizer.fit(train_df_corpus)

TfidfVectorizer(max_features=1600, ngram_range=(1, 3), stop_words='english',
                strip_accents='unicode', sublinear_tf=True,
                token_pattern='\\w+')

In [166]:
X_train_tfidf = tfidf_vectorizer.transform(train_df_corpus)

In [167]:
X_train_tfidf = X_train_tfidf.toarray()

In [168]:
tfidf_model.fit(X_train_tfidf,y_train)

LogisticRegression(multi_class='multinomial')

In [169]:
y_pred_train_tfidf = tfidf_model.predict_proba(X_train_tfidf)

In [170]:
log_loss(y_train,y_pred_train_tfidf)

1.0234542308030783

#### handling the test text sample data

In [171]:
test_text_df.sort_values(['ID']).head()

,ID,Text
0,0,2. This mutation resulted in a myeloproliferat...
9,9,Introduction The epidermal growth factor recep...
22,22,Peutz-Jeghers syndrome (PJS) is an autosomal d...
32,32,Individuals with neurofibromatosis type 1 (NFl...
37,37,The majority of human pancreatic cancers have ...


In [172]:
test_text_df.shape

(567, 2)

In [173]:
test_text_df['cleaned_text'] = test_text_df['Text'].map(cleanse_data,na_action='ignore')

In [174]:
test_text_df['cleaned_text'] = test_text_df['cleaned_text'].map(reduce_words)

In [175]:
test_text_df['cleaned_text_vector'] = test_text_df['cleaned_text'].map(word_count)

In [185]:
test_text_df['sorted_cleaned_text_vector'] = test_text_df['cleaned_text'].progress_map(freq_dist)

my bar!: 100%|██████████| 567/567 [00:02<00:00, 218.00it/s]


In [177]:
test_text_df = test_text_df.drop(['Text'],axis=1)

In [178]:
test_text_df.head()

,ID,cleaned_text,cleaned_text_vector,sorted_cleaned_text_vector
3767,3767,It Fl C329Y Deletions cells P525L R175H-expres...,"[4, 1, 81, 2, 41, 1, 1, 81, 98, 22, 14, 2, 5, ...","[98, 86, 81, 71, 63, 41, 32, 29, 29, 28, 28, 2..."
3185,3185,Aurora Aur-A required In cells Aur-A levels Au...,"[1, 122, 17, 30, 56, 122, 21, 122, 3, 3, 122, ...","[122, 74, 56, 54, 37, 32, 30, 30, 28, 26, 24, ..."
1291,1291,Aggressive mastocytosis ASM neoplasms tumors G...,"[1, 5, 13, 10, 2, 10, 25, 2, 44, 2, 81, 44, 25...","[81, 58, 46, 44, 43, 35, 25, 25, 21, 18, 17, 1..."
2155,2155,Mutations sequence-specific p53 transactivatio...,"[6, 6, 196, 73, 4, 35, 10, 9, 5, 135, 10, 11, ...","[196, 135, 73, 52, 45, 44, 44, 35, 35, 32, 30,..."
4114,4114,Introduction Somatic mutations cytosolic isoci...,"[1, 3, 63, 6, 13, 14, 64, 6, 10, 63, 2, 7, 1, ...","[71, 64, 63, 59, 53, 40, 35, 30, 30, 28, 27, 2..."


In [179]:
# get the text samples as list
test_corpus = test_text_df['cleaned_text'].tolist()

In [186]:
# transform the test case to get bag of words
test_bag_of_words_train = bow_vectorizer.fit_transform(test_corpus)

In [187]:
# convert the dictionary to an array
test_bag_of_words_array = test_bag_of_words_train.toarray()

In [188]:
test_bag_of_words_array.shape

(567, 1600)

In [189]:
# predict test using bag of word model
y_pred_test_bow = bow_model.predict_proba(test_bag_of_words_array)

In [190]:
y_pred_test_bow[0]

array([1.83112397e-12, 5.63882669e-08, 2.63455153e-10, 9.99999943e-01,
       2.38159074e-12, 9.00477562e-13, 8.86667915e-11, 3.67747652e-10,
       2.24606803e-12])

In [191]:
y_pred_test_df = pd.DataFrame(y_pred_test_bow)

In [201]:
test_text_df['ID'].shape

(567,)

In [209]:
y_pred_test_df['ID']= test_text_df['ID'].reset_index(drop=True)

In [210]:
y_pred_test_df.head()

,0,1,2,3,4,5,6,7,8,ID
0,1.831124e-12,5.638827e-08,2.634552e-10,9.999999e-01,2.381591e-12,9.004776e-13,8.866679e-11,3.677477e-10,2.246068e-12,3767
1,2.473665e-06,1.980595e-08,5.975855e-08,9.626488e-03,9.277821e-08,9.795628e-01,1.080688e-02,6.487810e-07,5.296784e-07,3185
2,3.125816e-01,1.552898e-02,3.578678e-05,5.772322e-01,2.885136e-05,2.503541e-02,5.936631e-02,6.191307e-03,3.999575e-03,1291
3,5.092804e-03,1.798601e-08,8.113812e-06,7.244426e-01,2.139201e-07,1.356692e-05,2.981770e-03,2.674463e-01,1.464205e-05,2155
4,1.396434e-10,9.817598e-01,1.639126e-06,1.169662e-10,3.641553e-10,2.342000e-06,6.847253e-05,1.816266e-02,5.095384e-06,4114


In [ ]:
# predict test using tfidf model
y_pred_test_tfidf = tfidf_model.predict_proba(test_bag_of_words_array)

In [214]:
stage2_test_text_df = pd.read_csv('stage2_test_text.csv', sep="\|\|", encoding="utf-8",engine="python", skiprows=1, names=["ID", "Text"])


In [215]:
stage2_test_text_df.shape

(986, 2)

In [218]:
stage2_test_text_df['cleaned_text'] = stage2_test_text_df['Text'].map(cleanse_data,na_action='ignore')

In [219]:
stage2_test_text_df['cleaned_text'] = stage2_test_text_df['cleaned_text'].map(reduce_words)

In [236]:
stage2_test_text_df.shape

(986, 3)

In [229]:
stage2_cleaned_view =pd.DataFrame({"Text":stage2_test_text_df['Text'].map(lambda x:len(x)),"cleaned_text":stage2_test_text_df['cleaned_text'].map(lambda x:len(x))})
stage2_cleaned_view.head()

,Text,cleaned_text
0,33403,14418
1,66400,29135
2,58544,26808
3,42023,17996
4,22499,8311


In [230]:
stage2_test_corpus = stage2_test_text_df['cleaned_text'].tolist()

In [239]:
X_stage2_test = bow_vectorizer.fit_transform(stage2_test_corpus).toarray()

In [240]:
X_stage2_test.shape

(986, 1600)

In [241]:
y_pred_stage2 = bow_model.predict_proba(X_stage2_test)

In [242]:
y_pred_stage2.shape

(986, 9)

In [246]:
y_pred_stage2_df = pd.DataFrame(y_pred_stage2)

In [247]:
y_pred_stage2_df['ID']=stage2_test_text_df['ID']

In [249]:
y_pred_stage2_df.columns

Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 'ID'], dtype='object')

In [254]:
y_pred_stage2_df.columns = ['class1','class2','class3','class4','class5','class6','class7','class8','class9','ID']

In [256]:
y_pred_stage2_df=y_pred_stage2_df[['ID','class1','class2','class3','class4','class5','class6','class7','class8','class9']]

,ID,class1,class2,class3,class4,class5,class6,class7,class8,class9
0,4.441999e-07,1,1.229218e-03,2.643612e-08,6.263733e-06,1.572336e-07,9.971541e-01,4.896264e-10,1.609409e-03,3.792160e-07
1,1.780477e-08,2,7.998047e-05,1.491813e-11,3.798138e-11,9.998514e-01,9.822595e-09,6.855978e-05,2.918746e-20,1.515444e-11
2,2.631865e-05,3,9.352904e-01,1.683814e-04,2.721056e-04,3.362818e-04,5.650345e-02,7.389802e-03,8.426276e-07,1.246086e-05
3,1.990002e-09,4,1.118231e-12,1.844340e-13,9.787649e-01,2.309602e-08,1.658664e-08,1.775307e-07,2.123488e-02,1.425437e-09
4,1.758151e-02,5,8.519845e-02,6.887932e-01,1.002284e-02,6.489397e-02,1.458021e-02,5.002989e-02,3.479671e-02,3.410322e-02
5,4.005247e-07,6,3.484074e-06,1.590991e-08,8.705694e-01,1.364901e-07,4.725819e-06,7.727982e-07,1.294182e-01,2.788608e-06
6,5.966151e-05,7,1.238184e-04,3.517886e-03,3.647401e-04,3.693967e-03,7.009471e-04,9.911654e-01,3.401021e-04,3.343870e-05
7,4.872153e-02,8,1.657041e-01,1.460685e-01,2.092281e-02,3.959692e-01,7.564039e-02,8.918721e-02,7.675469e-03,5.011076e-02
8,1.752501e-04,9,8.695033e-04,2.392063e-05,6.722958e-03,8.301005e-04,1.171525e-02,4.907042e-02,1.501870e-02,9.155739e-01
9,5.642489e-06,10,1.310976e-04,1.312335e-02,1.345899e-04,9.860797e-01,4.826678e-04,4.001257e-05,6.869909e-08,2.908710e-06


In [252]:
y_pred_stage2_df.to_csv("y_pred_stage2.csv")

In [ ]:
df=pd.DataFrame(columns=["Gene","Class","Variation","cleaned_text","cleaned_freq"])
for i in range(19):    
    df_new=class_9[class_9['cleaned_text'] == class_9['cleaned_text'].unique()[i]]
    df=pd.concat([df,df_new])

we have sorted the frequency dictionary based on values using the key parameter of sorted and higher values first using reverse

In [ ]:
sorted(test_dic.items(),key=lambda x:x[1], reverse=True)

In [ ]:
for i in range(10):
    print(scan_data1[i])

In [ ]:
cd /content

In [ ]:
pa_tree=ET.parse('pa2021.xml')

In [ ]:
root=pa_tree.getroot()

In [ ]:
# for child in root:
#   print(child.tag)

In [ ]:
for element_1 in root:
  #print("---------level_1-----------")
  #print(element_1.tag)
  
  for element_2 in element_1:
    #print("---------level_2-----------")
    #print("    "+element_2.tag," ",element_2.text)
    
    for element_3 in element_2:
      #print("---------level_3-----------")
      #print("        "+element_3.tag," ",element_3.text)
      
      for element_4 in element_3:
        #print("---------level_4-----------")
        if element_4.tag=="String":
          print("                   "+element_4.tag," ",element_4.text)
          
  #break
    

In [ ]:
len(root)

In [ ]:
desc_tree=ET.parse('desc2021.xml')

In [ ]:
desc_root=desc_tree.getroot()

In [ ]:
len(desc_root)

In [ ]:
term_description=[]
for element_1 in desc_root:
  #print("-------level_1----------")
  #print(element_1.tag," ",element_1.text)
  for element_2 in element_1:
    #print("    ----------level_2----------")
    #print("    "+element_2.tag," ",element_2.text)
    for element_3 in element_2:
      if element_3.tag=='String':
        #print("        ---------level_3--------")
        #print("        "+element_3.tag," ",element_3.text)
        term_description.append(element_3.text)

  #break

In [ ]:
len(term_description)

In [ ]:
term_description[:10]

In [ ]:
import csv 
with open('term_description.csv', 'w') as f: 
    write = csv.writer(f) 
    write.writerow(term_description) 
    

There are 2 ways to remove punctuation. one is to use the string library punctuation module on the text before tokenization, the other is to use the nltk.punct library

In [ ]:
len(scan_data1[1])

In [ ]:
#nltk.word_tokenize(scan_data1[1])

In [ ]:
# import string punctuation
import string
string.punctuation

In [ ]:
len(cleanup_text(scan_data1[1]))

In [ ]:
scan_data